In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import numpy as np
import pandas as pd
import scipy.stats as spst
import scipy.integrate as spint

from scipy.optimize import newton

import matplotlib.pyplot as plt
import warnings
import os
#import sys
#sys.path.insert(sys.path.index('')+1, 'D:/Github/PyFeng')
import pyfeng as pf

# Basket Option

In [19]:
sigma = 0.4*np.ones(4)
spot = np.ones(4)*100
texp=5

In [21]:
m = pf.BsmNdMc(sigma, cor=0.5, rn_seed=1234)
m.simulate(tobs = [texp], n_path=10000)
print(m.n_path)
m.path.shape

10000


(1, 10000, 4)

In [22]:
_ = m.simulate(tobs = [texp], n_path=50000, store=2)
print(m.n_path)
print(m.path.shape)

60000
(1, 60000, 4)


In [23]:
# varying strikes
payoff = lambda x: np.fmax(np.mean(x, axis=1) - strike, 0)

strikes = np.arange(50, 151, 10)
price = []
for strike in strikes:
    price.append(m.price_european(spot, texp, payoff))
np.array(price)

array([54.92679384, 48.11095395, 42.16350828, 36.99504987, 32.50356789,
       28.61957621, 25.26155084, 22.34144434, 19.79993616, 17.58970694,
       15.66817728])

In [24]:
# varying forwards
payoff = lambda x: np.fmax(np.mean(x, axis=1) - strike, 0)
strike = 100
price = []
for spot in np.arange(50, 151, 10):
    price.append(m.price_european(spot, texp, payoff))
np.array(price)

array([ 4.54551749,  7.79239206, 11.9092721 , 16.82164738, 22.42497892,
       28.61957621, 35.33937465, 42.51139716, 50.06198584, 57.93137459,
       66.08222223])

In [8]:
# varying except sigma1=100%
strike = spot = 100
payoff = lambda x: np.fmax(np.mean(x, axis=1) - strike, 0)
price = []
for sigma1 in np.array([5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 100])/100:
    sigma = sigma1 * np.ones(4)
    sigma[0] = 1
    #print(sigma)
    m = pf.BsmNdMc(sigma, cor=0.5, rn_seed=1234)
    m.simulate(tobs = [texp], n_path=400000)
    price.append(m.price_european(spot, texp, payoff))
np.array(price)

array([18.94495855, 20.45829493, 22.49840945, 24.87785108, 30.11378007,
       35.57949546, 41.05473026, 46.42608331, 51.61861158, 56.59050047,
       65.9322128 ])

# Simpon's method for integration

Let's integrate exp(-a*t) from 0 to texp

int exp(-a*t) = (1/a)(1 - exp(-a*texp))

In [29]:
a = 10
texp = 2
exact = (1 - np.exp(-a*texp))/a
exact

0.09999999979388464

In [31]:
def f(t, a):
    return np.exp(-a*t)

n_step = 100
tobs = np.arange(0, n_step+1)/n_step * texp
simp = spint.simps(f(tobs, a), dx=texp/n_step)
#simp = spint.simps(f(tobs, a), dx=1) * texp/n_step
print(simp, simp-exact)

0.10000088446767286 8.846737882123312e-07


# For SABR Model
## Integration of sigma(t)

In [32]:
# You can use BsmNdMc because sigma_t is also a geometric Brownian motion

vov = 0.3
texp = 5
m = pf.BsmNdMc(vov, rn_seed=1234)
tobs = np.arange(0, 101)/100*texp

In [35]:
_ = m.simulate(tobs = tobs, n_path=1000)

In [37]:
print(m.path.shape)
sigma_path = np.squeeze(m.path)
print(sigma_path.shape)

(101, 1000, 1)
(101, 1000)


In [40]:
sigma_final = sigma_path[-1,:]

In [42]:
int_var = spint.simps(sigma_path**2, dx=1, axis=0)/100
int_var.shape

(1000,)

In [43]:
np.mean(sigma_final)

1.0000568192728598